In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime
from sqlalchemy import create_engine
from config import password

In [2]:
# pull in DJIA data csv
file = "Resources/dow_copy.csv"
djia_df = pd.read_csv(file)
djia_df

,Date,Price,Open,High,Low,Vol.,Change %
0,12-Mar-12,"12,959.71","12,920.58","12,976.36","12,919.98",100.00M,0.29%
1,9-Jul-12,"12,736.29","12,772.02","12,772.02","12,686.57",100.15M,-0.28%
2,27-Dec-12,"13,096.31","13,114.97","13,141.74","12,964.08",100.16M,-0.14%
3,21-Jul-15,"17,919.29","18,096.67","18,096.67","17,868.34",100.17M,-1.00%
4,20-Apr-16,"18,096.27","18,059.49","18,167.63","18,031.21",100.21M,0.24%
...,...,...,...,...,...,...,...
2419,16-Sep-15,"16,739.95","16,599.51","16,755.98","16,593.90",99.62M,0.84%
2420,7-Dec-15,"17,730.51","17,845.49","17,845.49","17,639.25",99.67M,-0.66%
2421,12-Sep-13,"15,300.64","15,327.14","15,345.32","15,283.26",99.76M,-0.17%
2422,8-Feb-13,"13,992.97","13,944.05","14,022.62","13,944.05",99.86M,0.35%


In [3]:
#convert strings to lower case for PG Admin connection
djia_df.columns = djia_df.columns.str.lower()
djia_df.head()

,date,price,open,high,low,vol.,change %
0,12-Mar-12,"12,959.71","12,920.58","12,976.36","12,919.98",100.00M,0.29%
1,9-Jul-12,"12,736.29","12,772.02","12,772.02","12,686.57",100.15M,-0.28%
2,27-Dec-12,"13,096.31","13,114.97","13,141.74","12,964.08",100.16M,-0.14%
3,21-Jul-15,"17,919.29","18,096.67","18,096.67","17,868.34",100.17M,-1.00%
4,20-Apr-16,"18,096.27","18,059.49","18,167.63","18,031.21",100.21M,0.24%


In [4]:
#rename columns 
djia_df = djia_df.rename(columns={'vol.': 'vol_m', 'change %':'change_percent'})
djia_df.head()

,date,price,open,high,low,vol_m,change_percent
0,12-Mar-12,"12,959.71","12,920.58","12,976.36","12,919.98",100.00M,0.29%
1,9-Jul-12,"12,736.29","12,772.02","12,772.02","12,686.57",100.15M,-0.28%
2,27-Dec-12,"13,096.31","13,114.97","13,141.74","12,964.08",100.16M,-0.14%
3,21-Jul-15,"17,919.29","18,096.67","18,096.67","17,868.34",100.17M,-1.00%
4,20-Apr-16,"18,096.27","18,059.49","18,167.63","18,031.21",100.21M,0.24%


In [5]:
#remove commonas, M, and % prior to float conversion
djia_df.replace(',','', regex=True, inplace=True)
djia_df['vol_m'].replace('M','', regex=True, inplace=True)
djia_df.replace('%','', regex=True, inplace=True)

In [6]:
djia_df.head()

,date,price,open,high,low,vol_m,change_percent
0,12-Mar-12,12959.71,12920.58,12976.36,12919.98,100.00,0.29
1,9-Jul-12,12736.29,12772.02,12772.02,12686.57,100.15,-0.28
2,27-Dec-12,13096.31,13114.97,13141.74,12964.08,100.16,-0.14
3,21-Jul-15,17919.29,18096.67,18096.67,17868.34,100.17,-1.00
4,20-Apr-16,18096.27,18059.49,18167.63,18031.21,100.21,0.24


In [7]:
#check data types (need to convert object to float)
djia_df.dtypes

date              object
price             object
open              object
high              object
low               object
vol_m             object
change_percent    object
dtype: object

In [8]:
#convert all numbers to float
djia_df['price'] = djia_df['price'].astype(float)
djia_df['open'] = djia_df['open'].astype(float)
djia_df['high'] = djia_df['high'].astype(float)
djia_df['low'] = djia_df['low'].astype(float)
djia_df['vol_m'] = djia_df['vol_m'].astype(float)
djia_df['change_percent'] = djia_df['change_percent'].astype(float)

In [9]:
#recheck data types
djia_df.dtypes

date               object
price             float64
open              float64
high              float64
low               float64
vol_m             float64
change_percent    float64
dtype: object

In [10]:
#show database
djia_df.head()

,date,price,open,high,low,vol_m,change_percent
0,12-Mar-12,12959.71,12920.58,12976.36,12919.98,100.00,0.29
1,9-Jul-12,12736.29,12772.02,12772.02,12686.57,100.15,-0.28
2,27-Dec-12,13096.31,13114.97,13141.74,12964.08,100.16,-0.14
3,21-Jul-15,17919.29,18096.67,18096.67,17868.34,100.17,-1.00
4,20-Apr-16,18096.27,18059.49,18167.63,18031.21,100.21,0.24


In [11]:
# pull in store data csv
file2 = "Resources/train_v.csv"
store_df = pd.read_csv(file2)
store_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/6/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [12]:
#change all strings to lowercase and remove spaces to match PG Admin
store_df.columns = store_df.columns.str.lower()
store_df.columns = store_df.columns.str.replace(' ','_')
store_df.columns = store_df.columns.str.replace('-','_')
store_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,1,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/6/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [13]:
#select columns for analysis
store_df = store_df[['order_id', 'order_date', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'sub_category', 'product_name', 'sales']]
store_df.head()

,order_id,order_date,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,CA-2017-152156,8/11/2017,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,CA-2017-152156,8/11/2017,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,CA-2017-138688,12/6/2017,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,US-2016-108966,11/10/2016,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,US-2016-108966,11/10/2016,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [14]:
#put date info in date time format
djia_df['date']= pd.to_datetime(djia_df['date'])
djia_df.head()

,date,price,open,high,low,vol_m,change_percent
0,2012-03-12,12959.71,12920.58,12976.36,12919.98,100.00,0.29
1,2012-07-09,12736.29,12772.02,12772.02,12686.57,100.15,-0.28
2,2012-12-27,13096.31,13114.97,13141.74,12964.08,100.16,-0.14
3,2015-07-21,17919.29,18096.67,18096.67,17868.34,100.17,-1.00
4,2016-04-20,18096.27,18059.49,18167.63,18031.21,100.21,0.24


In [15]:
#put store info in date time format
store_df['order_date']= pd.to_datetime(store_df["order_date"], format='%m/%d/%Y', errors='coerce')
store_df.head()

,order_id,order_date,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,CA-2017-152156,2017-08-11,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,CA-2017-152156,2017-08-11,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,CA-2017-138688,2017-12-06,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,US-2016-108966,2016-11-10,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,US-2016-108966,2016-11-10,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [16]:
#check data types
store_df.dtypes

order_id                object
order_date      datetime64[ns]
city                    object
state                   object
postal_code            float64
region                  object
product_id              object
category                object
sub_category            object
product_name            object
sales                  float64
dtype: object

In [17]:
#connect to PG Admin
connection_String_format= '<user>:<passowrd>@<url>:<port>/<database_name>'
rds_connection_string = "postgres:"+{password}"@localhost:5432/ETL_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [18]:
#print out table names to check connection
engine.table_names()

['dow', 'store']

In [19]:
#load dow csv table into PG Admin
djia_df.to_sql(name='dow', con=engine, if_exists='append', index=False)

In [20]:
#read dow table in PG Admin to confirm load
pd.read_sql_query('select * from dow', con=engine).tail()

,id,date,price,open,high,low,vol_m,change_percent
2419,2420,2015-09-16,16739.95,16599.51,16755.98,16593.90,99.62,0.84
2420,2421,2015-12-07,17730.51,17845.49,17845.49,17639.25,99.67,-0.66
2421,2422,2013-09-12,15300.64,15327.14,15345.32,15283.26,99.76,-0.17
2422,2423,2013-02-08,13992.97,13944.05,14022.62,13944.05,99.86,0.35
2423,2424,2015-11-04,17867.58,17929.58,17964.12,17828.83,99.89,-0.28


In [21]:
#load store csv table into PG Admin
store_df.to_sql(name='store', con=engine, if_exists='append', index=False)

In [22]:
#read store table in PG Admin to confirm load
pd.read_sql_query('select * from store', con=engine).tail()

,id,order_id,order_date,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
9795,9796,CA-2017-125920,2017-05-21,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.798
9796,9797,CA-2016-128608,2016-12-01,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.368
9797,9798,CA-2016-128608,2016-12-01,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.188
9798,9799,CA-2016-128608,2016-12-01,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.376
9799,9800,CA-2016-128608,2016-12-01,Toledo,Ohio,43615.0,East,TEC-AC-10000487,Technology,Accessories,SanDisk Cruzer 4 GB USB Flash Drive,10.384
